In [7]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing

from fastai.tabular.all import *

import seaborn as sns
import glob

In [8]:
import warnings
warnings.filterwarnings('ignore')

## Data Gathering

In [9]:
comp_name = 'predict-west-nile-virus'
path = Path(f'/home/harish3110/Desktop/kaggle/{comp_name}')
path.mkdir(exist_ok=True)

dset_path = (path/'input')
dset_path.mkdir(exist_ok=True)

In [10]:
dep_var = 'WnvPresent'

In [11]:
dset_path = Path(f'/home/harish3110/Desktop/kaggle/{comp_name}/input')

train = pd.read_csv(f'{dset_path}/train.csv')
test = pd.read_csv(f'{dset_path}/test.csv')
weather = pd.read_csv(f'{dset_path}/weather.csv')
spray = pd.read_csv(f'{dset_path}/spray.csv')

ss = pd.read_csv(f'{dset_path}/sampleSubmission.csv')

In [12]:
len(test), len(ss)

(116293, 116293)

In [13]:
train.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634, USA",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634, USA",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0


In [14]:
test.head()

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634, USA",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
1,2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634, USA",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
2,3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634, USA",CULEX PIPIENS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
3,4,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634, USA",CULEX SALINARIUS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
4,5,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634, USA",CULEX TERRITANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9


In [15]:
weather.T

,0,1,2,3,4,5,6,7,8,9,...,2934,2935,2936,2937,2938,2939,2940,2941,2942,2943
Station,1,2,1,2,1,2,1,2,1,2,...,1,2,1,2,1,2,1,2,1,2
Date,2007-05-01,2007-05-01,2007-05-02,2007-05-02,2007-05-03,2007-05-03,2007-05-04,2007-05-04,2007-05-05,2007-05-05,...,2014-10-27,2014-10-27,2014-10-28,2014-10-28,2014-10-29,2014-10-29,2014-10-30,2014-10-30,2014-10-31,2014-10-31
Tmax,83,84,59,60,66,67,66,78,66,66,...,77,79,68,66,49,49,51,53,47,49
Tmin,50,52,42,43,46,48,49,51,53,54,...,51,54,45,48,36,40,32,37,33,34
Tavg,67,68,51,52,56,58,58,M,60,60,...,64,67,57,57,43,45,42,45,40,42
Depart,14,M,-3,M,2,M,4,M,5,M,...,16,M,10,M,-4,M,-4,M,-6,M
DewPoint,51,51,42,42,40,40,41,42,38,39,...,51,52,38,40,32,34,34,35,25,29
WetBulb,56,57,47,47,48,50,50,50,49,50,...,58,59,47,48,40,42,40,42,33,36
Heat,0,0,14,13,9,7,7,M,5,5,...,1,0,8,8,22,20,23,20,25,23
Cool,2,3,0,0,0,0,0,M,0,0,...,0,2,0,0,0,0,0,0,0,0


In [16]:
spray.head()

,Date,Time,Latitude,Longitude
0,2011-08-29,6:56:58 PM,42.391623,-88.089163
1,2011-08-29,6:57:08 PM,42.391348,-88.089163
2,2011-08-29,6:57:18 PM,42.391022,-88.089157
3,2011-08-29,6:57:28 PM,42.390637,-88.089158
4,2011-08-29,6:57:38 PM,42.390410,-88.088858


In [17]:
ss.head()

,Id,WnvPresent
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


## Data Preprocessing

### Handling dates

In [18]:
train = add_datepart(train, 'Date', drop=False)
test = add_datepart(test, 'Date', drop=False)
weather = add_datepart(weather, 'Date', drop=False)
spray = add_datepart(spray, 'Date', drop=False)

In [19]:
train.columns

Index(['Date', 'Address', 'Species', 'Week', 'Block', 'Street', 'Trap',
       'AddressNumberAndStreet', 'Latitude', 'Longitude', 'AddressAccuracy',
       'NumMosquitos', 'WnvPresent', 'Year', 'Month', 'Day', 'Dayofweek',
       'Dayofyear', 'Is_month_end', 'Is_month_start', 'Is_quarter_end',
       'Is_quarter_start', 'Is_year_end', 'Is_year_start', 'Elapsed'],
      dtype='object')

In [20]:
train.head().T

,0,1,2,3,4
Date,2007-05-29 00:00:00,2007-05-29 00:00:00,2007-05-29 00:00:00,2007-05-29 00:00:00,2007-05-29 00:00:00
Address,"4100 North Oak Park Avenue, Chicago, IL 60634, USA","4100 North Oak Park Avenue, Chicago, IL 60634, USA","6200 North Mandell Avenue, Chicago, IL 60646, USA","7900 West Foster Avenue, Chicago, IL 60656, USA","7900 West Foster Avenue, Chicago, IL 60656, USA"
Species,CULEX PIPIENS/RESTUANS,CULEX RESTUANS,CULEX RESTUANS,CULEX PIPIENS/RESTUANS,CULEX RESTUANS
Week,22,22,22,22,22
Block,41,41,62,79,79
Street,N OAK PARK AVE,N OAK PARK AVE,N MANDELL AVE,W FOSTER AVE,W FOSTER AVE
Trap,T002,T002,T007,T015,T015
AddressNumberAndStreet,"4100 N OAK PARK AVE, Chicago, IL","4100 N OAK PARK AVE, Chicago, IL","6200 N MANDELL AVE, Chicago, IL","7900 W FOSTER AVE, Chicago, IL","7900 W FOSTER AVE, Chicago, IL"
Latitude,41.9547,41.9547,41.995,41.9741,41.9741
Longitude,-87.801,-87.801,-87.7693,-87.8248,-87.8248


In [21]:
# Reference: https://www.kaggle.com/abhishek/vote-me-up

# Not using codesum for this benchmark
weather = weather.drop('CodeSum', axis=1)

# Split station 1 and 2 and join horizontally
weather_stn1 = weather[weather['Station']==1]
weather_stn2 = weather[weather['Station']==2]
weather_stn1 = weather_stn1.drop('Station', axis=1)
weather_stn2 = weather_stn2.drop('Station', axis=1)
weather = weather_stn1.merge(weather_stn2, on='Date')

In [22]:
train = train.merge(weather, on='Date')
test = test.merge(weather, on='Date')

In [24]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10506 entries, 0 to 10505
Data columns (total 89 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date                    10506 non-null  datetime64[ns]
 1   Address                 10506 non-null  object        
 2   Species                 10506 non-null  object        
 3   Week                    10506 non-null  UInt32        
 4   Block                   10506 non-null  int64         
 5   Street                  10506 non-null  object        
 6   Trap                    10506 non-null  object        
 7   AddressNumberAndStreet  10506 non-null  object        
 8   Latitude                10506 non-null  float64       
 9   Longitude               10506 non-null  float64       
 10  AddressAccuracy         10506 non-null  int64         
 11  NumMosquitos            10506 non-null  int64         
 12  WnvPresent              10506 non-null  int64 

### Categorical Variables Transform

In [25]:
unnecessary_cols = ['Address', 'AddressNumberAndStreet','NumMosquitos', 'Date', dep_var]

features = train.drop(unnecessary_cols, axis=1)
outcome = train[dep_var]

In [26]:
le = preprocessing.LabelEncoder()

In [27]:
cont,cat = cont_cat_split(features, 1, dep_var=dep_var)

In [28]:
cat

['Species',
 'Week',
 'Street',
 'Trap',
 'Is_month_end',
 'Is_month_start',
 'Is_quarter_end',
 'Is_quarter_start',
 'Is_year_end',
 'Is_year_start',
 'Elapsed',
 'Week_x',
 'Tavg_x',
 'Depart_x',
 'WetBulb_x',
 'Heat_x',
 'Cool_x',
 'Sunrise_x',
 'Sunset_x',
 'Depth_x',
 'Water1_x',
 'SnowFall_x',
 'PrecipTotal_x',
 'StnPressure_x',
 'SeaLevel_x',
 'AvgSpeed_x',
 'Is_month_end_x',
 'Is_month_start_x',
 'Is_quarter_end_x',
 'Is_quarter_start_x',
 'Is_year_end_x',
 'Is_year_start_x',
 'Elapsed_x',
 'Week_y',
 'Tavg_y',
 'Depart_y',
 'WetBulb_y',
 'Heat_y',
 'Cool_y',
 'Sunrise_y',
 'Sunset_y',
 'Depth_y',
 'Water1_y',
 'SnowFall_y',
 'PrecipTotal_y',
 'StnPressure_y',
 'SeaLevel_y',
 'AvgSpeed_y',
 'Is_month_end_y',
 'Is_month_start_y',
 'Is_quarter_end_y',
 'Is_quarter_start_y',
 'Is_year_end_y',
 'Is_year_start_y',
 'Elapsed_y']

In [29]:
features[cat] = features[cat].apply(le.fit_transform)

In [30]:
features.head().T

,0,1,2,3,4
Species,2.0,3.0,3.0,2.0,3.0
Week,0.0,0.0,0.0,0.0,0.0
Block,41.0,41.0,62.0,79.0,79.0
Street,32.0,32.0,27.0,109.0,109.0
Trap,1.0,1.0,6.0,13.0,13.0
...,...,...,...,...,...
Is_quarter_end_y,0.0,0.0,0.0,0.0,0.0
Is_quarter_start_y,0.0,0.0,0.0,0.0,0.0
Is_year_end_y,0.0,0.0,0.0,0.0,0.0
Is_year_start_y,0.0,0.0,0.0,0.0,0.0


## Sklearn Approach

### Splitting Dataset

In [31]:
X_train, X_test, y_train, y_test = train_test_split(features, outcome, test_size=0.3, random_state=2)

### DT

In [32]:
dt = DecisionTreeClassifier(random_state=2).fit(X_train, y_train)

### Bagged DT

In [33]:
def fit_dt_bagged(X, y):
    model = BaggingClassifier(base_estimator=dt, random_state=2)
    params = {'n_estimators':list(np.arange(1, 101, 2))}
    grid = GridSearchCV(model, params, scoring='recall', cv=3)
    grid_fit = grid.fit(X, y)

    # Return the optimal model after fitting the data
    return grid_fit.best_estimator_

# model_dt = fit_dt_bagged(X_train, y_train)

# # Produce the value for 'n_neighbors'
# print(model_dt.get_params())

In [34]:
bagged_dt = BaggingClassifier(base_estimator=dt, random_state=2, n_estimators=1000, n_jobs=-1,).fit(X_train, y_train)

### Random Forests

In [35]:
def fit_rf(X, y):
    model = RandomForestClassifier(random_state=2)
    params = {'n_estimators':list(np.arange(1, 101, 2))}
    grid = GridSearchCV(model, params, scoring='recall', cv=3)
    grid_fit = grid.fit(X, y)

    # Return the optimal model after fitting the data
    return grid_fit.best_estimator_

# model_rf = fit_rf(X_train, y_train)

# # Produce the value for 'n_neighbors'
# print(model_rf.get_params())

In [36]:
rf = RandomForestClassifier(random_state=2, n_estimators=1000, n_jobs=-1).fit(X_train, y_train)

### Comparing the models using cross validation:

In [37]:
# from sklearn.model_selection import KFold
# accuracy = []
# for index, model in enumerate([dt, bagged_dt, rf]): 
#     print(f'Model {index+1}:')
# #     kf = KFold(n_splits=10, shuffle=True, random_state=2)
#     scores = cross_val_score(estimator=model,
#                              X=features,
#                              y=outcome,
#                              cv=5,
#                              n_jobs=-1, 
#                              scoring='recall')
#     print(f'CV accuracy: {scores}')
#     print(f'CV accuracy: {np.mean(scores):.3f} +/- {np.std(scores):.3f}')
#     print(f'Bias Error: {(np.mean(1 - scores))}')
#     print(f'Variance Error: {np.var(scores)}')
#     print()
#     accuracy.append(scores)

In [38]:
# least_bias_err = 1
# least_variance_err = 1

# for index, acc in enumerate(accuracy):
#     if ((np.mean(1 - acc)) < least_bias_err):
#         least_bias_err = (np.mean(1 - acc))
#         index_bias = index
#     if (np.var(acc) < least_variance_err):
#         least_variance_err = np.var(acc)
#         index_var = index

# print(f'Model {index_bias + 1} has least bias: {least_bias_err}')
# print(f'Model {index_var + 1} has least variance: {least_variance_err}')

## Kaggle Submission

In [39]:
cols = ['Address', 'AddressNumberAndStreet', 'Date', 'Id']

In [40]:
test.drop(cols, axis=1, inplace=True)

In [41]:
cont,cat = cont_cat_split(test, 1)

In [42]:
test[cat] = test[cat].apply(le.fit_transform)

In [ ]:
# create predictions and submission file
predictions = rf.predict_proba(test)[:,1]

In [44]:
ss['WnvPresent'] = predictions
ss.to_csv('sklearn_baseline.csv', index=False)

That submission got a leaderboard score of 0.69220 on the LB